In [ ]:
# Adapted from LLama Factory
# https://colab.research.google.com/drive/1eRTPn37ltBbYsISy9Aw2NuI2Aq5CQrD9?usp=sharing#scrollTo=CS0Qk5OR0i4Q

In [2]:
!which python
!which pip

~/.conda/envs/py38/bin/python
~/.conda/envs/py38/bin/pip


In [18]:
%cd /research/rgs01/home/clusterHome/jpastr08/biohackathon/KIDS24-team12/vm_files/Jose
%pwd

/research/rgs01/home/clusterHome/jpastr08/biohackathon/KIDS24-team12/vm_files/Jose


'/research/rgs01/home/clusterHome/jpastr08/biohackathon/KIDS24-team12/vm_files/Jose'

In [19]:
# Commented out to be able to `Run All Cells` smoothly.

#!git clone --depth 1 https://github.com/PepeRulo/LLaMA-Factory.git

#!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
#!pip uninstall -y jax
#!pip install -e .[torch,bitsandbytes,liger-kernel]
#!pip install --upgrade huggingface_hub

In [20]:
import sys
import torch

print(sys.version)
print(torch.__version__)

3.8.20 | packaged by conda-forge | (default, Sep 30 2024, 17:52:49) 
[GCC 13.3.0]
2.3.1+cu121


In [21]:
import psutil

# Get the total RAM in bytes
total_ram = psutil.virtual_memory().total

# Convert bytes to GB
total_ram_gb = total_ram / (1024 ** 3)

print(f"Total RAM: {total_ram_gb:.2f} GB")


Total RAM: 1507.35 GB


In [22]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")

device = torch.device("cuda")

# Get the name of the GPU
gpu_name = torch.cuda.get_device_name(device)
print(f"GPU: {gpu_name}")

# You can also get more details
print(f"Number of GPUs: {torch.cuda.device_count()}")
print(f"Current CUDA device: {torch.cuda.current_device()}")
print(f"CUDA capability: {torch.cuda.get_device_capability(device)}")

print(torch.__version__)
print(torch.version.cuda)

!nvidia-smi

GPU: NVIDIA A100-SXM4-80GB
Number of GPUs: 4
Current CUDA device: 0
CUDA capability: (8, 0)
2.3.1+cu121
12.1
Thu Dec  5 20:16:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:0B:00.0 Off |                    0 |
| N/A   40C    P0              65W / 400W |      5MiB / 81920MiB |      0%      Default |
|                                         |      

In [23]:
import os
from huggingface_hub import login
with open("hf_token.txt", 'r') as file:
    token = file.readline().strip()
    login(token=token)

In [27]:
%cd /research/rgs01/home/clusterHome/jpastr08/biohackathon/KIDS24-team12/vm_files/Jose/LLaMA-Factory

/research/rgs01/home/clusterHome/jpastr08/biohackathon/KIDS24-team12/vm_files/Jose/LLaMA-Factory


In [25]:
%cp /research/rgs01/home/clusterHome/jpastr08/biohackathon/KIDS24-team12/vm_files/Jose/files/dataset.json \
    ./data/dragen_alpaca.json

In [29]:
import json
with open("data/dataset_info.json", "r") as f:
    datasets = json.load(f)
    
# Per https://github.com/hiyouga/LLaMA-Factory/blob/main/data/README.md#alpaca-format
if "dragen_alpaca" not in datasets:
    datasets["dragen_alpaca"] = {
        "file_name": "dragen_alpaca.json",
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "response": "output",
            "system": "system",
            "history": "history"
        }
    }
    
    with open("data/dataset_info.json", "w") as f:
        json.dump(datasets, f, indent=2, ensure_ascii=False)

In [ ]:
# Resources:
# https://www.reddit.com/r/LocalLLaMA/comments/1fps1cp/llama32_vs_llama31_in_medical_domain_llama31_70b/
# https://github.com/NVIDIA/RULER?tab=readme-ov-file